In [199]:
import pandas as pd
import geopandas as gpd
import numpy as np
# import plotly as px
from pyproj import Transformer
import requests

In [200]:
weer_api = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\apidata_2021.csv")
weer= pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\weerstations_noordholland.csv", sep= ";")
weer = gpd.GeoDataFrame(weer, geometry=gpd.GeoSeries.from_wkt(weer['WKT']))
weer['STN']=weer['STN_y']
weer.drop(columns=['WKT', 'STN_x', 'STN_y'], inplace=True)
hectointervallen = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\hectointervallen.txt") 
hectointervallen = hectointervallen[hectointervallen.nunique()[hectointervallen.nunique() != 1].index]
hectopunten = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\hectopunten.txt") 
hectopunten = hectopunten[hectopunten.nunique()[hectopunten.nunique() != 1].index]
juncties = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\juncties.txt") 
juncties = juncties[juncties.nunique()[juncties.nunique() != 1].index]
puntlocaties = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\puntlocaties.txt") 
puntlocaties = puntlocaties[puntlocaties.nunique()[puntlocaties.nunique() != 1].index]
ongevallen = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Ongevallengegevens\ongevallen.txt", encoding="ISO-8859-15")
ongevallen = ongevallen[ongevallen.nunique()[ongevallen.nunique() != 1].index]
ongevallen.drop(columns=['PVOPGEM', 'WVL_ID','WVG_ID', 'WVG_AN', 'WDK_ID', 'WDK_AN',
                         'LGD_ID', 'BZD_ID_VM1','BZD_ID_VM2', 'BZD_ID_VM3', 'BZD_VM_AN', 'BZD_ID_IF1', 'BZD_ID_IF2',
                         'BZD_ID_IF3', 'BZD_IF_AN', 'BZD_ID_TA1', 'BZD_ID_TA2', 'BZD_ID_TA3', 'HUISNUMMER'], inplace=True)


C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3399033372.py:14: DtypeWarning: Columns (1,24,39) have mixed types. Specify dtype option on import or set low_memory=False.
  ongevallen = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Ongevallengegevens\ongevallen.txt", encoding="ISO-8859-15")


In [201]:
samengevoegd_df = ongevallen.merge(juncties, on='JTE_ID', how='left')
samengevoegd_df['FK_VELD5']=samengevoegd_df['FK_VELD5_y']
samengevoegd_df = samengevoegd_df.merge(puntlocaties, on='FK_VELD5', how='left')
samengevoegd_df = samengevoegd_df.merge(puntlocaties, on='FK_VELD5', how='left')
df= samengevoegd_df
samengevoegd_df = df[df.nunique()[df.nunique() != 1].index]
samengevoegd_df.drop(columns=['AP4_CODE', 'AP5_CODE', 'FK_VELD5_y', 'X_COORD_y', 'Y_COORD_y', 
                              'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'DATUM_VKL', 'DAG_CODE', 'MND_NUMMER',
                              'TIJDSTIP', 'UUR', 'DDL_ID', 'DAGTYPE', 'WEEKNR',
                              'ANTL_SEH','ANTL_GOV','ANTL_TDT', 'MNE_CODE', 'ZAD_ID', 'WGD_CODE_2',
                              'IND_ALC'], inplace=True)

df= samengevoegd_df
# df.columns
df2=df[['X_COORD_x', 'Y_COORD_x', 'VKL_NUMMER', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID', 'BEBKOM', 'WGD_CODE_1']]
df2['X']=df2['X_COORD_x']
df2['Y']=df2['Y_COORD_x']
df2.drop(columns=['X_COORD_x', 'Y_COORD_x'], inplace=True)

C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3343418178.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['X']=df2['X_COORD_x']
C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3343418178.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Y']=df2['Y_COORD_x']
C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3343418178.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [202]:
samengevoegd_df.columns

Index(['VKL_NUMMER', 'REGNUMMER', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID',
       'NIVEAUKOP', 'WSE_ID', 'BEBKOM', 'MAXSNELHD', 'WGD_CODE_1', 'JTE_ID',
       'WVK_ID', 'HECTOMETER', 'FK_VELD5_x', 'GME_ID', 'GME_NAAM', 'PVE_CODE',
       'PVE_NAAM', 'KDD_NAAM', 'PLT_NAAM', 'DIENSTCODE', 'DIENSTNAAM',
       'DISTRCODE', 'DISTRNAAM', 'WBRSRT_R', 'WBRSRT_P', 'WBRSRT_G',
       'WBRSRT_W', 'WBRSRT_T', 'ANTL_TAK', 'ACTUEEL', 'SLE_TYPE', 'SLE_NUMMER',
       'FK_VELD5', 'X_COORD_x', 'Y_COORD_x'],
      dtype='object')

In [203]:
trans = Transformer.from_crs("ETRS89", "EPSG:4326", always_xy=True)
xx, yy = trans.transform(df2["X"].values, df2["Y"].values)

df2["X"] = xx
df2["Y"] = yy
df2["X"] = (df2["X"]/64000)+3
df2["Y"] = (df2["Y"]/110000)+48
# df2.drop(columns=['X_COORD_x', 'Y_COORD_x'], inplace=True)

df2=df2[['X', 'Y', 'VKL_NUMMER', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID', 'BEBKOM', 'WGD_CODE_1']]
df2.dropna()
gdf = gpd.GeoDataFrame(
    df2, geometry=gpd.points_from_xy(df2.X, df2.Y))
gdf.dropna()

C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3967659008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["X"] = xx
C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3967659008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Y"] = yy
C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\3967659008.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,X,Y,VKL_NUMMER,AP3_CODE,ANTL_PTJ,AOL_ID,BEBKOM,WGD_CODE_1,geometry
8,5.698422,51.700464,20210000435,DOD,2,7,BI,D,POINT (5.69842 51.70046)
16,5.440999,51.761055,20210000567,DOD,2,7,BU,D,POINT (5.44100 51.76105)
17,6.486673,52.904554,20210000568,DOD,2,7,BU,D,POINT (6.48667 52.90455)
25,4.427013,51.588056,20210072804,UMS,2,7,BI,D,POINT (4.42701 51.58806)
29,6.990462,53.006926,20210074980,UMS,2,7,BI,D,POINT (6.99046 53.00693)
...,...,...,...,...,...,...,...,...,...
113144,5.783843,51.445289,20210027300,UMS,3,7,BU,D,POINT (5.78384 51.44529)
113147,6.821636,53.241292,20210027330,UMS,2,0,BI,D,POINT (6.82164 53.24129)
113150,4.806394,52.453405,20210027591,UMS,3,9,BI,D,POINT (4.80639 52.45340)
113151,4.558489,52.098595,20210027600,UMS,3,7,BI,D,POINT (4.55849 52.09860)


In [204]:
weer_api

,Unnamed: 0,datetime,Windsnelheid (km/h),Windrichting (Graden),Luchtdruk (ps),Zichtbaarheid (Decimeter ver kunnen kijken),Regenval (mm/h),station_id,Temperatuur (C),Weercode
0,0,2021-01-01 00:00:00+00:00,0.80,232.2,1005.84,123.0,0.0,6260,-2.1,0.0
1,1,2021-01-01 00:10:00+00:00,0.90,233.2,1005.84,126.0,0.0,6260,-2.0,0.0
2,2,2021-01-01 00:20:00+00:00,0.57,233.5,1005.94,109.0,0.0,6260,-2.2,0.0
3,3,2021-01-01 00:30:00+00:00,0.39,233.1,1005.94,137.0,0.0,6260,-2.3,0.0
4,4,2021-01-01 00:40:00+00:00,0.59,232.1,1005.94,142.0,0.0,6260,-2.3,0.0
...,...,...,...,...,...,...,...,...,...,...
559279,559279,2021-12-30 23:20:00+00:00,13.65,231.3,1012.47,8510.0,0.0,6242,10.5,0.0
559280,559280,2021-12-30 23:30:00+00:00,13.24,230.8,1012.57,8590.0,0.0,6242,10.5,0.0
559281,559281,2021-12-30 23:40:00+00:00,13.11,231.4,1012.37,8720.0,0.0,6242,10.6,0.0
559282,559282,2021-12-30 23:50:00+00:00,13.20,230.2,1012.37,8880.0,0.0,6242,10.5,0.0


In [205]:
weer = pd.merge(weer, weer_api, on='station_id')
weer.drop(columns=['fid', 'Unnamed: 0', 'NAME'], inplace=True)


In [206]:
# weer['datetime'] = pd.to_datetime(weer['datetime'])
# 

In [207]:

weer['datetime'] = pd.to_datetime(weer['datetime'], format='%Y-%m-%d %H:%M:%S')

weer['maand'] = weer['datetime'].dt.month
weer['dag'] = weer['datetime'].dt.day


In [208]:
weer['m/d/STN'] =weer['maand'].astype(str) + '/' + weer['dag'].astype(str)+ '/' + weer['STN'].astype(str)


In [209]:
weer.columns

Index(['LON(east)', 'LAT(north)', 'ALT(m)', 'geometry', 'station_id', 'STN',
       'datetime', 'Windsnelheid (km/h)', 'Windrichting (Graden)',
       'Luchtdruk (ps)', 'Zichtbaarheid (Decimeter ver kunnen kijken)',
       'Regenval (mm/h)', 'Temperatuur (C)', 'Weercode', 'maand', 'dag',
       'm/d/STN'],
      dtype='object')

In [210]:
weer2=weer.groupby('m/d/STN')["Windsnelheid (km/h)", 'Windrichting (Graden)', 'Luchtdruk (ps)', 'Zichtbaarheid (Decimeter ver kunnen kijken)', 
                         'Regenval (mm/h)','Temperatuur (C)', 'Weercode', 'STN'].median()

C:\Users\tjibb\AppData\Local\Temp\ipykernel_8312\216662622.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weer2=weer.groupby('m/d/STN')["Windsnelheid (km/h)", 'Windrichting (Graden)', 'Luchtdruk (ps)', 'Zichtbaarheid (Decimeter ver kunnen kijken)',


In [224]:
weer3 = weer[['STN', 'geometry']]
weer3

,STN,geometry
0,260,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1..."
1,260,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1..."
2,260,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1..."
3,260,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1..."
4,260,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1..."
...,...,...
559279,242,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1..."
559280,242,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1..."
559281,242,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1..."
559282,242,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1..."


In [226]:
weer3 = weer3.drop_duplicates()


In [227]:
weer3

,STN,geometry
0,260,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1..."
50844,240,"MULTIPOLYGON (((5.02839 52.28670, 5.02820 52.2..."
101688,225,"MULTIPOLYGON (((4.58437 52.27886, 4.58451 52.2..."
152532,209,"MULTIPOLYGON (((4.52338 52.32138, 4.51242 52.3..."
203376,257,"MULTIPOLYGON (((4.56887 52.49745, 4.56924 52.4..."
254220,248,"MULTIPOLYGON (((5.27671 52.68508, 5.26938 52.6..."
305064,249,"MULTIPOLYGON (((5.06081 52.57997, 5.07205 52.5..."
355908,258,"MULTIPOLYGON (((5.37670 52.76665, 5.37769 52.7..."
406752,267,"MULTIPOLYGON (((5.13328 53.04158, 5.13740 53.0..."
457596,235,"MULTIPOLYGON (((4.63317 52.75539, 4.63342 52.7..."


In [228]:
weer4 = pd.merge(weer2, weer3, on='STN')

In [229]:
weer_omtemergen= weer4

In [230]:
weer_omtemergen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4015 entries, 0 to 4014
Data columns (total 9 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   Windsnelheid (km/h)                          4015 non-null   float64 
 1   Windrichting (Graden)                        4015 non-null   float64 
 2   Luchtdruk (ps)                               4015 non-null   float64 
 3   Zichtbaarheid (Decimeter ver kunnen kijken)  4015 non-null   float64 
 4   Regenval (mm/h)                              4015 non-null   float64 
 5   Temperatuur (C)                              4015 non-null   float64 
 6   Weercode                                     4015 non-null   float64 
 7   STN                                          4015 non-null   float64 
 8   geometry                                     4015 non-null   geometry
dtypes: float64(8), geometry(1)
memory usage: 313.7 KB


In [231]:
gdf = gdf.dropna()

In [232]:
weer4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4015 entries, 0 to 4014
Data columns (total 9 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   Windsnelheid (km/h)                          4015 non-null   float64 
 1   Windrichting (Graden)                        4015 non-null   float64 
 2   Luchtdruk (ps)                               4015 non-null   float64 
 3   Zichtbaarheid (Decimeter ver kunnen kijken)  4015 non-null   float64 
 4   Regenval (mm/h)                              4015 non-null   float64 
 5   Temperatuur (C)                              4015 non-null   float64 
 6   Weercode                                     4015 non-null   float64 
 7   STN                                          4015 non-null   float64 
 8   geometry                                     4015 non-null   geometry
dtypes: float64(8), geometry(1)
memory usage: 313.7 KB


In [236]:
weer_omtemergen = gpd.GeoDataFrame(weer_omtemergen, geometry=(weer_omtemergen['geometry']))

In [237]:
merged_gdf = gpd.sjoin(gdf, weer_omtemergen, predicate='within')


In [238]:
merged_gdf

,X,Y,VKL_NUMMER,AP3_CODE,ANTL_PTJ,AOL_ID,BEBKOM,WGD_CODE_1,geometry,index_right,Windsnelheid (km/h),Windrichting (Graden),Luchtdruk (ps),Zichtbaarheid (Decimeter ver kunnen kijken),Regenval (mm/h),Temperatuur (C),Weercode,STN
394,4.771971,52.730472,20210080706,LET,2,1,BI,R,POINT (4.77197 52.73047),730,1.79000,238.60,1007.49,49900.0,0.0,4.3,0.0,235.0
398,4.778672,52.738382,20210080712,UMS,1,9,BU,D,POINT (4.77867 52.73838),730,1.79000,238.60,1007.49,49900.0,0.0,4.3,0.0,235.0
497,4.807741,52.936414,20210081452,UMS,4,7,BI,D,POINT (4.80774 52.93641),730,1.79000,238.60,1007.49,49900.0,0.0,4.3,0.0,235.0
563,4.812609,52.758818,20210082114,UMS,2,7,BI,D,POINT (4.81261 52.75882),730,1.79000,238.60,1007.49,49900.0,0.0,4.3,0.0,235.0
570,4.811406,52.742245,20210082121,UMS,3,4,BI,D,POINT (4.81141 52.74225),730,1.79000,238.60,1007.49,49900.0,0.0,4.3,0.0,235.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41852,4.527688,52.462267,20210034058,UMS,2,7,BI,D,POINT (4.52769 52.46227),364,4.11427,199.55,0.00,0.0,0.0,0.0,0.0,209.0
53741,4.517625,52.360518,20210085188,LET,2,7,BU,D,POINT (4.51762 52.36052),364,4.11427,199.55,0.00,0.0,0.0,0.0,0.0,209.0
56104,4.526193,52.323866,20210066720,UMS,2,7,BU,D,POINT (4.52619 52.32387),364,4.11427,199.55,0.00,0.0,0.0,0.0,0.0,209.0
62122,4.513559,52.337518,20210064168,LET,2,7,BU,R,POINT (4.51356 52.33752),364,4.11427,199.55,0.00,0.0,0.0,0.0,0.0,209.0
